In [1]:
import mysql.connector as mysql
import database as db
import csv, sys
import image_generator
import new_feature_extractor
import numpy as np
import os
import cv2
import math
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
import sklearn.preprocessing as preprocessing
from sklearn import tree
from sklearn import metrics

In [2]:
data = pd.read_csv("feature_values.csv") #My generated file with normalized and scaled values
data.head()

,filename,Mobilenet_v1,Inception_v4,Resnet_v1_152,kp_surf,average_perceived_brightness,contrast,area_by_perim,aspect_ratio,edge_length1,hue1
0,ILSVRC2012_val_00000001.JPEG,1,0,0,3657,189.692869,24.824576,33.795181,3.882353,31,0.007221
1,ILSVRC2012_val_00000002.JPEG,0,0,0,7029,174.428835,58.779438,52.382199,3.152174,70,0.002446
2,ILSVRC2012_val_00000003.JPEG,1,0,0,2248,93.423358,44.725204,87.616262,0.980226,302,0.017275
3,ILSVRC2012_val_00000004.JPEG,1,0,0,1211,150.763832,42.931180,75.765422,1.289963,267,0.019652
4,ILSVRC2012_val_00000005.JPEG,0,1,0,6270,118.078176,57.030755,91.372124,1.232628,134,0.012794


In [3]:
features = list(data.drop(['Mobilenet_v1', 'Inception_v4', 'Resnet_v1_152', 'filename'], axis = 1).columns)
datasets = []

In [4]:
for x in features:
    datasets.append(data.sort_values(by=[x], ascending=False).head(15))

In [5]:
datasets

[                          filename  Mobilenet_v1  Inception_v4  Resnet_v1_152  \
 1921  ILSVRC2012_val_00001922.JPEG             1             0              0   
 2732  ILSVRC2012_val_00002733.JPEG             1             0              0   
 217   ILSVRC2012_val_00000218.JPEG             1             0              0   
 519   ILSVRC2012_val_00000520.JPEG             1             0              0   
 745   ILSVRC2012_val_00000746.JPEG             1             0              0   
 2941  ILSVRC2012_val_00002942.JPEG             1             0              0   
 2622  ILSVRC2012_val_00002623.JPEG             1             0              0   
 1693  ILSVRC2012_val_00001694.JPEG             1             0              0   
 1987  ILSVRC2012_val_00001988.JPEG             1             0              0   
 739   ILSVRC2012_val_00000740.JPEG             1             0              0   
 2765  ILSVRC2012_val_00002766.JPEG             1             0              0   
 3585  ILSVRC201

In [6]:
result = pd.concat(datasets)

In [8]:
result.head(45)

,filename,Mobilenet_v1,Inception_v4,Resnet_v1_152,kp_surf,average_perceived_brightness,contrast,area_by_perim,aspect_ratio,edge_length1,hue1
1921,ILSVRC2012_val_00001922.JPEG,1,0,0,10000,110.017537,61.518410,106.138693,2.241042,231,0.012337
2732,ILSVRC2012_val_00002733.JPEG,1,0,0,10000,130.709982,62.888649,186.741506,0.871571,697,0.027023
217,ILSVRC2012_val_00000218.JPEG,1,0,0,10000,114.880617,51.910461,353.833739,1.283558,637,0.027157
519,ILSVRC2012_val_00000520.JPEG,1,0,0,10000,113.484961,72.001143,345.707258,0.638670,502,0.019248
745,ILSVRC2012_val_00000746.JPEG,1,0,0,10000,118.179975,53.447065,244.609733,1.694087,656,0.025411
2941,ILSVRC2012_val_00002942.JPEG,1,0,0,10000,98.481370,79.834035,415.846154,0.962264,1685,0.019468
2622,ILSVRC2012_val_00002623.JPEG,1,0,0,10000,138.539865,75.020716,225.930181,1.173238,894,0.018426
1693,ILSVRC2012_val_00001694.JPEG,1,0,0,10000,112.096161,59.189542,174.857854,1.164615,315,0.015612
1987,ILSVRC2012_val_00001988.JPEG,1,0,0,10000,133.061725,61.639186,120.910359,1.472906,90,0.022033
739,ILSVRC2012_val_00000740.JPEG,1,0,0,10000,107.386505,62.677643,245.247917,1.868526,639,0.014167


In [9]:
indexes = list(result.index.values)

In [10]:
indexes

[1921,
 2732,
 217,
 519,
 745,
 2941,
 2622,
 1693,
 1987,
 739,
 2765,
 3585,
 997,
 1616,
 596,
 3063,
 2710,
 4383,
 3570,
 2247,
 650,
 2852,
 2974,
 1733,
 265,
 2532,
 84,
 117,
 4453,
 450,
 2848,
 283,
 256,
 1297,
 4302,
 4980,
 685,
 2180,
 2031,
 557,
 1494,
 2368,
 1779,
 2144,
 358,
 2208,
 596,
 1053,
 4421,
 2941,
 4386,
 4689,
 217,
 519,
 1815,
 2419,
 997,
 1616,
 1631,
 1584,
 493,
 4856,
 1826,
 1294,
 1766,
 1322,
 4540,
 2036,
 1097,
 2256,
 3427,
 1681,
 1526,
 1963,
 2566,
 2208,
 1815,
 2941,
 4386,
 1778,
 2765,
 2148,
 3507,
 732,
 1053,
 3540,
 4421,
 1616,
 1584,
 3878,
 295,
 3466,
 316,
 4607,
 1419,
 428,
 125,
 3249,
 33,
 2450,
 1027,
 3155,
 2735,
 2693,
 1659]